In [8]:
from pathlib import Path
import requests
import sklearn
import sklearn.datasets
import torch
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch import nn

from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import optim
import torch.nn.functional as F
from torch import nn
import numpy as np

C:\Users\CONNOR~1\AppData\Local\Temp/ipykernel_12616/3002970644.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y_data = dataset.target.values.to_numpy(dtype=np.int)


In [ ]:
dataset = sklearn.datasets.fetch_openml("CIFAR_10_small", cache=True)
X_data = dataset.data.values
Y_data = dataset.target.values.to_numpy(dtype=np.int)
x_train, x_valid, y_train, y_valid = train_test_split(X_data, Y_data, test_size= .25, random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)

x_train = torch.FloatTensor(x_train)
x_valid = torch.FloatTensor(x_valid)
y_train = torch.LongTensor(y_train)
y_valid = torch.LongTensor(y_valid)

#x_train, y_train, x_valid, y_valid = map(
#    torch.tensor, (x_train, y_train, x_valid, y_valid)
#)

n, c = x_train.shape
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())


bs = 64
epochs = 10
lr = .1
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

loss_func = F.cross_entropy

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

def preprocess(x, y):
    return x.view(-1, 3, 32, 32), y


class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)


train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

model = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=.9)
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

tensor([[ 53.,  50.,  47.,  ...,  69.,  44.,  86.],
        [229., 227., 227.,  ..., 188., 187., 187.],
        [248., 250., 236.,  ..., 235., 235., 235.],
        ...,
        [114., 131., 147.,  ...,  39.,  48.,  88.],
        [126., 130., 136.,  ...,  94.,  98.,  92.],
        [179., 153., 156.,  ..., 231., 233., 232.]], dtype=torch.float64) tensor([[ 84., 100., 112.,  ...,  96.,  91.,  85.],
        [ 98., 129., 123.,  ..., 105., 102., 107.],
        [ 78.,  88., 120.,  ...,  69.,  69.,  64.],
        ...,
        [  0.,   1.,   2.,  ..., 153., 101.,  48.],
        [205., 206., 205.,  ..., 104., 123., 145.],
        [  2.,   1.,   0.,  ..., 183., 184., 175.]], dtype=torch.float64)
torch.Size([15000, 3072])
tensor(0., dtype=torch.float64) tensor(255., dtype=torch.float64)


AssertionError: Size mismatch between tensors

In [ ]:
pred_list = []
sum = 0
for xb, yb in train_dl:
    ypred = model(xb)
    sum = sum + accuracy(ypred,yb)
acc = sum/len(train_dl)
print(acc)

